In [1]:
from lib import module
import time
import os, itertools
import pickle
from sklearn.metrics import f1_score, accuracy_score
import statistics
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold, train_test_split, RandomizedSearchCV, GridSearchCV
import pandas as pd
import numpy as np
from torch_geometric.data import Data
import os
import torch
import argparse
from tqdm import tqdm
import errno
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)


base_path = ''
feature_networks_integration = ['clinical', 'cna', 'exp','coe','met','mut'] # datatypes to concatanate node features from
learning_rate = 0.001
hid_size = 128

# optimize for optional feature selection of node features
feature_selection_per_network = [False, False, False,False, False, False]
top_features_per_network = [50, 50, 50,50,50,50]
optional_feat_selection = False
boruta_runs = 100
boruta_top_features = 50


# fixed
max_epochs = 500
min_epochs = 200
patience = 30

# fixed to get the same results from the tool each time
random_state = 404

# SUPREME run
print('SUPREME is setting up!')

dataset_name = 'full_data'

path = base_path + "data/" + dataset_name
if not os.path.exists(path):
    raise FileNotFoundError(errno.ENOENT, os.strerror(errno.ENOENT), path)
        
device = torch.device('cuda:1')

SUPREME is setting up!


In [2]:
criterion = torch.nn.CrossEntropyLoss()
data_path_node =  base_path + 'data/' + dataset_name +'/'
run_name = 'SUPREME_'+  dataset_name + '_results'
save_path = base_path + run_name + '/'

if not os.path.exists(base_path + run_name):
    os.makedirs(base_path + run_name + '/')

file = base_path + 'data/' + dataset_name +'/labels.pkl'
with open(file, 'rb') as f:
    labels = pickle.load(f)

file = base_path + 'data/' + dataset_name + '/mask_values.pkl'
if os.path.exists(file):
    with open(file, 'rb') as f:
        train_valid_idx, test_idx = pickle.load(f)
    print('use pre-defined split')
else:
    train_valid_idx, test_idx= train_test_split(np.arange(len(labels)), test_size=0.20, shuffle=True, stratify=labels)
    print('use random split')
start = time.time()

    
x_lists = []
for netw in feature_networks_integration:
    file = base_path + 'data/' + dataset_name +'/'+ netw +'.pkl'
    with open(file, 'rb') as f:
        feat = pickle.load(f)
        values = feat.values
        x_lists.append(values)
        print(values.shape)
new_x = torch.tensor(np.concatenate(x_lists,-1),dtype=torch.float32).to(device)


train_idx,valid_idx = train_test_split(train_valid_idx,test_size=0.2)
train_mask = np.array([i in set(train_idx) for i in range(new_x.shape[0])])
valid_mask = np.array([i in set(valid_idx) for i in range(new_x.shape[0])])
test_mask = np.array([i in set(test_idx) for i in range(new_x.shape[0])])
y_test = pd.DataFrame(labels[test_mask].cpu().numpy()).values.ravel()

in_size = new_x.shape[1]
out_size = torch.unique(labels).shape[0]

use pre-defined split
(1378, 14)
(1378, 1003)
(1378, 1046)
(1378, 49)
(1378, 1058)
(1378, 173)


In [8]:
new_x.shape


torch.Size([1378, 3343])